In [3]:
!pip uninstall -y bitsandbytes
!pip install bitsandbytes --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 282.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 192.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 294.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 186.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 285.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 232.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 170.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 240.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 182.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 372.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 220.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    U

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel
import torch
import os

lora_path = os.path.join("/content/drive", "My Drive", "ColabData", "tiny-finetuned")
base_model_name = "TinyLLaMA/TinyLLaMA-1.1B-Chat-v1.0"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    #load_in_4bit=True,
    #torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, lora_path)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_fea

In [3]:
system_prompt = (
    "You are an expert coder -- please identify the computer science concepts and explain the logic in this code block in less than 500 words."
)

code_snippet = "def twoSum(nums, target):\n    for i in range(len(nums)):\n        for j in range(i+1, len(nums)):\n            if nums[i] + nums[j] == target:\n                return [i, j]"
full_prompt = f"### Instruction:\n{system_prompt}\n\n{code_snippet}\n\n### Response:\n"


In [4]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # device=0,
    torch_dtype=torch.bfloat16
)

output = pipe(full_prompt, max_new_tokens=750, temperature=0.7, do_sample=True)

print("🧠 Model Output:")
print(output[0]['generated_text'].split("<|assistant|>")[-1].strip())


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

🧠 Model Output:
### Instruction:
You are an expert coder -- please identify the computer science concepts and explain the logic in this code block in less than 500 words.

def twoSum(nums, target):
    for i in range(len(nums)):
        for j in range(i+1, len(nums)):
            if nums[i] + nums[j] == target:
                return [i, j]

### Response:
{"key": "chatbox", "data": { "body": "### Computer Science Concepts\n\nThis code snippet is a simple function to find the two numbers in a given array that sum to a given target value. The function checks whether the target value is present in the array at any given index, and returns the index of the first found pair. This problem falls under the concept of \"two-sum\" problems.\n\nHere's a detailed explanation:\n\n```python\ndef twoSum(nums, target):\n    for i in range(len(nums))\n        for j in range(i+1, len(nums))\n            if nums[i] + nums[j] == target:\n                return [i, j]\n    return None\n```\n\n- `for i in r

In [5]:
merged_model_path = "/content/drive/My Drive/ColabData/tiny_merged_model_fixed"

# Merge the LoRA adapter into the base model
merged_model = model.merge_and_unload()  # This returns a full model!

# Save merged model
merged_model.save_pretrained(merged_model_path, safe_serialization=False)

# Save tokenizer too
tokenizer = AutoTokenizer.from_pretrained(lora_path, trust_remote_code=True)
tokenizer.save_pretrained(merged_model_path)

print(f"✅ Merged model saved to: {merged_model_path}")

✅ Merged model saved to: /content/drive/My Drive/ColabData/tiny_merged_model_fixed


In [8]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# === CONFIG ===
base_model_name = "TinyLLaMA/TinyLLaMA-1.1B-Chat-v1.0"
lora_path = os.path.join("/content/drive", "My Drive", "ColabData", "tiny-finetuned")
output_path = os.path.join("/content/drive", "My Drive", "ColabData", "clean_merged_model")

# === Load base model ===
print("📦 Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# === Load and merge LoRA ===
print("🔗 Loading LoRA adapter...")
model = PeftModel.from_pretrained(base_model, lora_path)
print("🧬 Merging LoRA into base model...")
merged_model = model.merge_and_unload()

# === Save merged model ===
print(f"💾 Saving merged model to {output_path}...")
merged_model.save_pretrained(output_path, safe_serialization=False)

# === Save tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.save_pretrained(output_path)

# === Validate contents ===
print("🔍 Verifying saved model...")
model_file = os.path.join(output_path, "pytorch_model.bin")
if os.path.exists(model_file):
    size_mb = os.path.getsize(model_file) / (1024 * 1024)
    print(f"✅ Model saved: {model_file} ({size_mb:.2f} MB)")
    if size_mb < 100:
        print("⚠️ Model size is suspiciously small. GGUF conversion may fail.")
    else:
        print("🎯 Model looks good for GGUF conversion.")
else:
    print("❌ Model file not found. Save may have failed.")

print("\n✅ Done! You can now run convert_hf_to_gguf.py on this folder.")


📦 Loading base model...
🔗 Loading LoRA adapter...
🧬 Merging LoRA into base model...
💾 Saving merged model to /content/drive/My Drive/ColabData/clean_merged_model...
🔍 Verifying saved model...
✅ Model saved: /content/drive/My Drive/ColabData/clean_merged_model/pytorch_model.bin (2098.24 MB)
🎯 Model looks good for GGUF conversion.

✅ Done! You can now run convert_hf_to_gguf.py on this folder.


In [7]:
!zip -r clean_merged_model.zip clean_merged_model

  adding: clean_merged_model/ (stored 0%)
  adding: clean_merged_model/tokenizer_config.json (deflated 68%)
  adding: clean_merged_model/pytorch_model.bin


zip error: Interrupted (aborting)
